# Real-NVPs

**Goal:** Implement a Real-NVP model on your own and evaluate the results on some **toy problem**.

![](flow-graphic.png)


**Table of Contents:**
1. [Dataset](#data) 
2. [Model architecture](#model)
- Q1: Implement the t (translation) NN in the `CouplingLayer` 
- Q2: Implement the forward function n the `CouplingLayer` 
- Q3: Implement the log(det(Jacobian)) for the reverse function
3. [Training](#train)
- Q4: Add the loss function definition to the `train` function
- Q5: Visualize the density of $f^{-1}(X)$ for a trained model
4. [Visualizations](#viz)
- Code already provided, just for fun to look at what the step-by-step flow denisty propogation is doing.
5. [Bonus: Conditional generative model](#condFlow)

## 1. Dataset
<a name="data"></a>

Let's use the crescent moons dataset as a nice 2d test bed example

In [ ]:
from sklearn.datasets import make_moons

import matplotlib.pyplot as plt


import math
import numpy as np
import torch
import torch.nn as nn

In [ ]:
nsamples = 30_000
noise = 0.05
X = make_moons(nsamples, noise=noise)[0]

In [ ]:
nViz=500

plt.scatter(X[:nViz,0], X[:nViz,1])

plt.xlabel('$X_0$',fontsize=18)
plt.ylabel('$X_1$',fontsize=18)

## 2. Model architecture
<a name="model"></a>

### Implementation detail: Masking

In the lecture, we talked about the affine transformations transforming a bipartite grouping of the variables (via "coupling layers").

I.e, for $f: x \rightarrow y, x,y \in \mathbb{R}^n$, we group the variables into the <span style="color:deeppink">transforming dimensions</span> and <span style="color:blue">conditioning dimensions</span>, and just transfom one set of variables at a time, while leaving the others stationary, e.g,

$$ y_{1:d} = x_{1:d} $$
$$y_{d+1:D} = x_{d+1:D} \cdot s(x_{1:d})  + t(x_{1:d})$$

(and in the next step of the flow $x_{1:d}$ would get transformed while $x_{d+1:D}$ would be useds for the conditioning).

As described in [1605.08803](https://arxiv.org/abs/1605.08803), it's common to implement the variable partitioning with a binary mask, here denoted $m$, where
$$
m_i = 
\begin{cases}
0, \quad \text{transforming dimension} \\
1, \quad \text{conditioning dimension}
\end{cases}
$$
This allows us to implement the the update eqs for a single flow step more succinctly as:

$$y =  m \odot x + (1-m) \odot \left[x \cdot s(m \odot x )  + t(m \odot x ) \right].$$



We simplify a little bit to try to get into a more succinct form.

- Write in terms of `log_s` (as this is the NN we have).

$$y =  m \odot x + (1-m) \odot \left[x \cdot \exp \log s(m \odot x )  + t(m \odot x ) \right]$$

- Group the terms with $x$ together.

$$y =  x \odot \left[ m  + (1-m) \cdot \exp \log s(m \odot x ) \right]  + t(m \odot x ) \cdot (1-m)$$

We want to take the `exp` outside of the expression in $[ \ldots ]$.

Note that

$$\exp(m_i) = 
1, \  \ \text{if} \ m_i = 0 .$$


$$y =  x \odot  \exp \left[( \log s(m \odot x ) \cdot (1-m)  \right]  + t(m \odot x ) \cdot (1-m)$$

In [ ]:
class CouplingLayer(nn.Module):
    """ An implementation of a coupling layer
    from RealNVP (https://arxiv.org/abs/1605.08803).
    """

    def __init__(self, num_inputs, num_hidden, mask):
        super(CouplingLayer, self).__init__()

        self.num_inputs = num_inputs
        self.mask = mask
            
        self.log_s_net = nn.Sequential(
            nn.Linear(num_inputs, num_hidden), nn.Tanh(),
            nn.Linear(num_hidden, num_hidden), nn.Tanh(),
            nn.Linear(num_hidden, num_inputs))
        
        '''
        TO DO (Q1): Setup the network for the shift of the affine transformation
        - Use 2 hidden layers
        - `num_hidden` units / hidden layer
        - ReLU activation
        '''
        self.t_net = 

    def forward(self, inputs,  mode='forward'):
        '''
        Returns a tuple with 
        - the transofrmation, f(inpts)
        - and the log Jacobian of f

        Whether we apply f or f^{-1} depends on whether we're running 
        with `mode` as forward or reverse
        '''
        mask = self.mask
        masked_inputs = inputs * mask
        
        if mode == 'forward':
            
            '''
            TO DO (Q2): Implement the transformation
            '''
            
            log_s = 
            s = 
            t = 
            out = 
            
            return out, log_s.sum(-1, keepdim=True)
                        
        else:
            
            log_s = self.log_s_net(masked_inputs) * (1 - mask)
            t = self.t_net(masked_inputs) * (1 - mask)
            s = torch.exp(log_s)
            
            '''
            TO DO (Q3): Implement the Jacobian
            '''
            log_jacob = 
            
            return (inputs - t) / s, log_jacob

### Sanity check

After working through steps (1) -- (3), the CouplingLayer class should work :) 

And the code block below let's you check the implementation.

![](frogs.png)

In [ ]:
# Generate a random normally distributed vector 
# with 100 samples in 2 dimensions
Z = torch.randn(100,2)

# dim 0 will be the conditioning variable
# dim 1 will be the transforming variable
mask = torch.Tensor([1,0])
f = CouplingLayer(2,64,mask)

with torch.no_grad():
    out, log_p = f(Z)

# The histogram shows the inputs and the outputs
fig, axes = plt.subplots(1,2,figsize=(8,4))

for i, ax, c in zip([0,1],axes,['royalblue','hotpink']):

    ax.hist(Z[:,i].numpy(),25,(-3,3),label=f'$Z_{i}$',color=c,alpha=.4)
    ax.hist(out[:,i].numpy(),25,(-3,3),label=f'$f(Z_{i})$',color=c,lw=2.5,histtype='step')
    
    ax.set_xlabel(f'dim {i}',fontsize=18)
    ax.legend(fontsize=12)
    
plt.show()

^ Is the result what you expected?

The `FlowSequential` then stitches together a sequence of CouplingLayers together to form a single model that can evaluate eitheer

For this tutorial we're giving a ready-to-go implementation `FlowSequential` class :)


In [ ]:
class FlowSequential(nn.Sequential):
    """ A sequential container for flows.
    In addition to a forward pass it implements a backward pass and
    computes log jacobians.
    """

    def forward(self, inputs, mode='forward'):
        """ Performs a forward or reverse pass for flow modules.
        Args:
            inputs: a tuple of inputs and logdets
            mode: to run direct computation or inverse
        """
        
        self.num_inputs = inputs.size(-1)

        # Initialize logdets to 0
        logdets = torch.zeros(inputs.size(0), 1, device=inputs.device)

        assert mode in ['forward', 'reverse']
        if mode == 'forward':
            for module in self._modules.values():
                inputs, logdet = module(inputs, mode)
                logdets += logdet
        else:
            for module in reversed(self._modules.values()):
                inputs, logdet = module(inputs, mode)
                logdets += logdet

        return inputs, logdets

    def log_probs(self, inputs):
        z, log_jacob = self(inputs,mode='reverse')
          
        log_probs = (-0.5 * z.pow(2) - 0.5 * math.log(2 * math.pi)).sum(-1, keepdim=True)
        return (log_probs + log_jacob).sum(-1, keepdim=True)

    def sample(self, num_samples=None):
        
        noise = torch.Tensor(num_samples, self.num_inputs).normal_()
        noise = noise.to(device)

        samples = self.forward(noise, mode='forward')[0]
        
        return samples

### Flow architecture

Let's consider a flow with 9 blocks and the $\log(s)$ and $t$ NNs with 64 hidden units per hidden layer.

In [ ]:
num_blocks=9
num_hidden=64

In [ ]:
# Check for a GPU
if torch.cuda.is_available():
    device = 'cuda:0'
else:
    device='cpu'
    
print(device)

In [ ]:
num_inputs = X.shape[1]

mask = torch.arange(0, num_inputs) % 2
mask = mask.to(device).float()

modules=[]

for _ in range(num_blocks):
    modules.append( CouplingLayer( num_inputs, num_hidden, mask) )
    mask = 1 - mask

In [ ]:
model = FlowSequential(*modules)

In [ ]:
model

## 4. Training
<a name="train"></a>

In [ ]:
X_torch = torch.Tensor(X.astype(np.float32))

In [ ]:
batch_size=256

nTrain=24_000
nVal=3_000

In [ ]:
# For comparison with a later cell block ... look at the density 
# of the _test data_ before training the odel

model = model.to(device)
with torch.no_grad():
    Z_test = model(X_torch[nTrain+nVal:].to(device),mode='reverse')[0].cpu().numpy()

plt.figure(figsize=(5,5))

plt.scatter(*Z_test.T)

plt.xlim(-3,3)
plt.ylim(-3,3)

plt.xlabel('$Z_0$',fontsize=20)
plt.ylabel('$Z_1$',fontsize=20)


In [ ]:
kwargs = {'num_workers': 4, 'pin_memory': True}  if torch.cuda.is_available() else {}

In [ ]:
train_loader = torch.utils.data.DataLoader(
    X_torch[:nTrain], batch_size=batch_size, shuffle=True,**kwargs)

valid_loader = torch.utils.data.DataLoader(
    X_torch[nTrain : nTrain+nVal],
    batch_size=batch_size,
    shuffle=False,
    drop_last=False,
    **kwargs)

test_loader = torch.utils.data.DataLoader(
    X_torch[nTrain+nVal :],
    batch_size=batch_size,
    shuffle=False,
    drop_last=False,
    **kwargs)

In [ ]:
def train(model, train_loader):
    
    model.train()
    train_loss = 0

    for batch_idx, data in enumerate(train_loader):
   
        data = data.to(device)
        optimizer.zero_grad()
        
        '''
        TO DO (Q4): Fill in the expression for the loss
        '''
        loss = 
        train_loss += loss.item()
        
        loss.backward()
        optimizer.step()
        
    train_loss /= len(train_loader)
    return train_loss

In [ ]:
def validate(model, loader, prefix='Validation'):
    
    model.eval()
    val_loss = 0

    for batch_idx, data in enumerate(loader):

        data = data.to(device)
        with torch.no_grad():
            val_loss += -model.log_probs(data).sum().item()  # sum up batch loss
     
    val_loss /= len(loader.dataset)
    return val_loss 

In [ ]:
lr=1e-4

optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-6)

In [ ]:
def draw_model(model, cond_input=None, title=''):
    '''
    Goal: Given the given model weights, show
    (1) The density
    (2) Samples from p_X(x)
    
    Inputs:
    - model that we're plotting the density of
    - cond_input: Extra logic for the conditional gen model
                  (bonus prob)
    - title: Super title over 2 subfigs
    '''
    
    fig, (ax1,ax2) = plt.subplots(1,2,figsize=(10,4),
                                  gridspec_kw={'hspace':10})

    # Title the plot with the log(p) on the validation set
    if title:
        fig.suptitle(title)

    '''
    (1) Plot the density
    '''
    x = np.linspace(-1,2)
    y = np.linspace(-.75,1.25)

    xx,yy = np.meshgrid(x,y)

    X_grid = np.vstack([xx.flatten(),yy.flatten()]).T.astype(np.float32)
    X_grid.T

    X_grid = torch.tensor(X_grid).to(device)

    if cond_input:
        # For last (bonus) prob
        y0,y1 = cond_input
        Y_grid = torch.ones_like(X_grid).to(device)
        Y_grid[:,0] = y0
        Y_grid[:,1] = y1
    
        with torch.no_grad():
            log_probs = model.log_probs(X_grid,Y_grid).cpu().numpy()

    else:
        with torch.no_grad():
            log_probs = model.log_probs(X_grid).cpu().numpy()

    ax1.pcolormesh(xx,yy,np.exp(log_probs.reshape(50,50)),shading='auto',cmap='coolwarm')

    ax1.set_xlabel('$X_0$',fontsize=12)
    ax1.set_ylabel('$X_1$',fontsize=12)

    '''
    (2) Plot samples from the model
    '''
    with torch.no_grad():
        
        if cond_input:
            # For last (bonus) prob
            Y_con = torch.ones(500,2).to(device)
            Y_con[:,0] = y0
            Y_con[:,1] = y1
            X_gen = model.sample(Y_con).cpu().numpy()
        else:
            X_gen = model.sample(500).cpu().numpy()
        
    ax2.scatter(*X_gen.T)

    ax2.set_xlabel('$X_0$',fontsize=12)
    ax2.set_ylabel('$X_1$',fontsize=12)

    ax2.set_xlim(x[[0,-1]])
    ax2.set_ylim(y[[0,-1]])

    plt.show()


In [ ]:
epochs=51

train_losses = np.zeros(epochs)
val_losses = np.zeros(epochs)


for i in range(epochs):
    
    train_losses[i] = train(model, train_loader)
    val_losses[i] = validate(model, valid_loader)

    print(f'Epoch {i}: train loss = {train_losses[i]:.4f}, val loss = {val_losses[i]:.4f}')
        
    if i % 10 == 0:
        # Draw the model prediction
        draw_model(model,title=f'Validation loss = {val_losses[i]}')
        


Plot the training and validation loss.

In [ ]:
plt.plot(train_losses,label='train')
plt.plot(val_losses,label='val')

plt.xlabel('epochs',fontsize=20)
plt.ylabel('Loss',fontsize=20)
plt.legend(fontsize=15)
plt.show()

**What's the density of the test data _after_ training?**

In [ ]:
'''
TO DO (Q5): Get the (2d) density of the test data
'''

with torch.no_grad():
    Z_test = 

plt.figure(figsize=(5,5))

plt.scatter(*Z_test.T)

plt.xlim(-3,3)
plt.ylim(-3,3)

plt.xlabel('$Z_0$',fontsize=20)
plt.ylabel('$Z_1$',fontsize=20)


Is this what we expect for the trained flow?

## 4. Vizualize the flow
<a name="viz"></a>

This was a 9 step flow, so let's sample some noise, and look at each of the 10 steps of the flow that goes into the crescent moon density :)

In [ ]:
N = 500
Z = torch.randn(N,2)

In [ ]:
cmask = np.where((Z[:,0]<0) & (Z[:,1] >0),0,
                np.where((Z[:,0]>0) & (Z[:,1] >0),1,
                        np.where((Z[:,0]>0) & (Z[:,1]<0),2,3)))

In [ ]:
colors = [f'C{i}' for i in cmask ]

In [ ]:
nrows = 2
ncols = 5

fig, ax = plt.subplots(nrows,ncols,figsize=(16,9))

ax[0,0].scatter(*Z.T, color=colors)
ax[0,0].set_title('Norm(0,1)',fontsize=14)

'''
Let each subsequent step of the flow viz the jazz
'''

Z_gpu = Z.float().to(device)

for k,module in enumerate(modules):
    
    with torch.no_grad():    
        Z_gpu, _ = module(Z_gpu)
    
    
    i = (k+1) // ncols
    j = (k+1) % ncols
    
    ax[i,j].set_title(f'Flow step {k+1}',fontsize=14)
    
    ax[i,j].scatter(*Z_gpu.cpu().numpy().T, color=colors)
      
plt.show()

# 5. Bonus: Conditional flow

Lots of applications in science involve conditional flow models, can you extend our the model we built here to be conditioned on the center point for the moon? 

**Plan:** 
- Train sampling moon density centers $y \in \mathbb{R}^2$ **uniformly** from [0,1]

Recall, we were training the 30k training samples for modelling 2 dimensions.

In [ ]:
nsamples,num_inputs

In [ ]:
Y = torch.Tensor(nsamples, num_inputs).uniform_()

In [ ]:
X_cond = X_torch + Y

**Let's look at the prediction for some slices of the conditional output**
- $0 < y_0 < .05$ and $0 < y_1 < .05$
- $0.5 < y_0 < .55$ and $.5 < y_1 < .55$
- $0.5 < y_0 < .55$ and $.5 < y_1 < .55$

In [ ]:
dy = .05
for i,y_min in enumerate([0,.5,1-dy]):
    
    y_max = y_min+dy
    mi = (Y[:,0] > y_min) & (Y[:,0] < y_max)
    mi = mi & (Y[:,1] > y_min) & (Y[:,1] < y_max)
    
    y_avg = .5 * (y_min + y_max)
    
    c=f'C{i}'
    plt.scatter(*X_cond[mi].T.numpy(),alpha=.5,color=c)
    plt.scatter([y_avg],[y_avg],250,marker='x',label='avg center',color=c)
    plt.xlabel('$X_0$',fontsize=18)
    plt.ylabel('$X_1$',fontsize=18)
    plt.legend()
    plt.title(f'$y_0,y_1$ center in ({y_min},{y_max})')
    
    plt.xlim(-1.2,3.2)
    plt.ylim(-.7,2.2)
    plt.show()
    

### TO DO: Implement the conditional flow model

**Hint 1:** Extend the coupling layer to a conditional coupling layer

In [ ]:
class CondCouplingLayer(nn.Module):

    def __init__(self, num_inputs, num_cond_inputs, num_hidden, mask):
        super(CondCouplingLayer, self).__init__()

        '''
        TO DO: Fill in 
        (Tip: look at `Coupling layer` for inspiration)
        '''

    def forward(self, inputs, cond_inputs,  mode='forward'):
        
        mask = self.mask
        masked_inputs = inputs * mask
        
        all_inputs = torch.cat([masked_inputs, cond_inputs],axis=1)
        
        if mode == 'forward':
            
            '''
            TO DO : Fill in 
            '''
            raise NotImplementedError  
                
        else:
            
            '''
            TO DO : Fill in 
            '''
            raise NotImplementedError  

**Hint 2:** Implement a `CondFlowSequential` class that calls the `CondCouplingLayer` class.

In [ ]:
class CondFlowSequential(nn.Sequential):
    """ A sequential container for flows extending the 
    FlowSequential class for conditional inputs
    """

    def forward(self, inputs, cond_inputs, mode='forward'):
        """ Performs a forward or reverse pass for flow modules.
        Args:
            inputs: a tuple of inputs and logdets
            cond_inputs: The conditional inputs to the flow
            mode: to run direct computation or inverse
        """
        raise NotImplementedError

    def log_probs(self, inputs, cond_inputs):
        raise NotImplementedError
        
    def sample(self, cond_inputs, num_samples=None):
        
        raise NotImplementedError

**Put the pieces together to define a flow**

In [ ]:
# Extending the code that we had before
num_cond_inputs = Y.shape[1]

modules=[]

for _ in range(num_blocks):
    modules.append( CondCouplingLayer( num_inputs, num_cond_inputs, num_hidden, mask) )
    mask = 1 - mask

In [ ]:
# To DO: Define the conditional flow from these modules
cond_flow = 

In [ ]:
# Once you've defined the model... put it on the gpu
cond_flow = cond_flow.to(device)

**Training code**

It's the same as before, just evaluating the conditional generative model, so we'll just give you the training functions.

We'll just use a `from torch.utils.data.Dataset` first to pass to the `DataLoaders` that we'll create :)

In [ ]:
'''
New dataset class to deal w/ the tuple of X,Y input
'''

from torch.utils.data import Dataset

class CondMoonsDataset(Dataset):
    '''
    Skeleton class taken from:
    https://pytorch.org/tutorials/beginner/basics/data_tutorial.html
    '''
    def __init__(self,nExamples,noise=0.05):
        
        X_np = make_moons(nExamples, noise=noise)[0]
        X = torch.Tensor(X_np).float()
            
        num_inputs = X.shape[1]
        Y = torch.Tensor(nExamples, num_inputs).uniform_()
         
        X = X+Y
            
        self.X = X
        self.Y = Y
        
    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]

In [ ]:
cond_train_loader = torch.utils.data.DataLoader(
    CondMoonsDataset(nTrain), 
    batch_size=batch_size, shuffle=True,**kwargs)

cond_valid_loader = torch.utils.data.DataLoader(
    CondMoonsDataset(nVal),
    batch_size=batch_size,
    shuffle=False,
    drop_last=False,
    **kwargs)

test_loader = torch.utils.data.DataLoader(
    CondMoonsDataset(nVal),
    batch_size=batch_size,
    shuffle=False,
    drop_last=False,
    **kwargs)

In [ ]:
def cond_train(model, train_loader, opt):
    
    model.train()
    train_loss = 0

    for batch_idx, (X,Y) in enumerate(train_loader):
        
        X = X.to(device) 
        Y = Y.to(device) 
        opt.zero_grad()
        
        loss = -model.log_probs(X,Y).mean()
        train_loss += loss.item()
        
        loss.backward()
        opt.step()
        
    train_loss /= len(train_loader)
    return train_loss

def cond_validate(model, loader, prefix='Validation'):
    
    model.eval()
    val_loss = 0

    for batch_idx, (X,Y) in enumerate(loader):

        X = X.to(device) 
        Y = Y.to(device) 
            
        with torch.no_grad():
            val_loss += -model.log_probs(X,Y).sum().item()  # sum up batch loss
     
    val_loss /= len(loader.dataset)
    return val_loss 

In [ ]:
opt = torch.optim.Adam(cond_flow.parameters(), lr=lr, weight_decay=1e-6)

epochs=51

cond_train_losses = np.zeros(epochs)
cond_val_losses = np.zeros(epochs)

for i in range(epochs):
    
    cond_train_losses[i] = cond_train(cond_flow, cond_train_loader, opt)
    cond_val_losses[i] = cond_validate(cond_flow, cond_valid_loader)

    print(f'Epoch {i}: train loss = {cond_train_losses[i]:.4f}, val loss = {cond_val_losses[i]:.4f}')
        
    if i % 10 == 0:
        # Draw the model prediction
        draw_model(cond_flow,cond_input=[0,0],
                   title=f'Validation loss = {cond_val_losses[i]}')
        

**Sanity check:** What have we learned about crescent moons in the three cases that we had above?

In [ ]:
fig, axs = plt.subplots(1,3,figsize=(12,3),
                            gridspec_kw={'wspace':.3})

fig.suptitle('Conditional flow')

# Define the density grid
x = np.linspace(-1.2,3.2)
y = np.linspace(-.7,2.2)

xx,yy = np.meshgrid(x,y)

X_grid = np.vstack([xx.flatten(),yy.flatten()]).T.astype(np.float32)
X_grid.T

X_grid = torch.tensor(X_grid).to(device)

'''
Loop over the centers
'''
for yi,ax,cmap in zip([0, 0.5, 1],axs,['Blues','Oranges','Greens']):
    
    # Plot the density
    
    Y_grid = torch.ones_like(X_grid).to(device)
    Y_grid[:,0] *= yi
    Y_grid[:,1] *= yi
    
    with torch.no_grad():
        log_probs = cond_flow.log_probs(X_grid,Y_grid).cpu().numpy()

    ax.pcolormesh(xx,yy,np.exp(log_probs.reshape(50,50)),shading='auto',cmap=cmap)

    ax.set_xlabel('$X_0$',fontsize=12)
    ax.set_ylabel('$X_1$',fontsize=12)

    ax.scatter([yi],[yi],300,marker='x',color='k')
    # break
plt.show()


**Resources:**
- This tutorial for the code in this repo came from the [pytorch-flows](https://github.com/ikostrikov/pytorch-flows) repo.
- The [nflows](https://github.com/bayesiains/nflows.git) is also a very nice package that includes the Real-NVP model and also the RQ-NSF that we also talked about in the lecture.